In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [2]:

!python -V

Python 3.9.19


In [4]:

import pickle
import pandas as pd

In [5]:

with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/ubuntu/.local/share/virtualenvs/web-service-mlflow-ZkaECBB5/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ubuntu/.local/share/virtualenvs/web-service-mlflow-ZkaECBB5/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [7]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)


In [12]:
from sklearn.metrics import mean_squared_error

In [13]:
target = 'duration'
y_train = df[target].values

mean_squared_error(y_train, y_pred, squared=False)

8.249440527525513

In [9]:
y_pred

array([16.24590642, 26.1347962 , 11.88426424, ..., 11.59533603,
       13.11317847, 12.89999218])

In [14]:
y_pred.std()

6.247488852238703

In [22]:
len(y_pred)

3316216

In [23]:
len(df)

3316216

In [10]:
X_val

<3316216x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6632424 stored elements in Compressed Sparse Row format>

In [17]:
year=2023
month=3
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [18]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration,ride_id
0,2,2023-03-01 00:06:43,2023-03-01 00:16:43,1.0,0.00,1.0,N,238,42,2,...,1.0,0.5,0.00,0.0,1.0,11.10,0.0,0.00,10.000000,2023/03_0
1,2,2023-03-01 00:08:25,2023-03-01 00:39:30,2.0,12.40,1.0,N,138,231,1,...,6.0,0.5,12.54,0.0,1.0,76.49,2.5,1.25,31.083333,2023/03_1
2,1,2023-03-01 00:15:04,2023-03-01 00:29:26,0.0,3.30,1.0,N,140,186,1,...,3.5,0.5,4.65,0.0,1.0,28.05,2.5,0.00,14.366667,2023/03_2
3,1,2023-03-01 00:49:37,2023-03-01 01:01:05,1.0,2.90,1.0,N,140,43,1,...,3.5,0.5,4.10,0.0,1.0,24.70,2.5,0.00,11.466667,2023/03_3
4,2,2023-03-01 00:08:04,2023-03-01 00:11:06,1.0,1.23,1.0,N,79,137,1,...,1.0,0.5,2.44,0.0,1.0,14.64,2.5,0.00,3.033333,2023/03_4


In [25]:
df_result = pd.DataFrame({"ride_id": df["ride_id"], "y_pred" : y_pred})

In [27]:
df_result.to_parquet(
    "./output_file",
    engine='pyarrow',
    compression=None,
    index=False
)

In [28]:
!ls -al

total 69388
drwxr-xr-x 5 ubuntu ubuntu     4096 Jun 24 02:51 .
drwxr-xr-x 4 ubuntu ubuntu     4096 Jun 23 23:57 ..
drwxr-xr-x 2 ubuntu ubuntu     4096 Jun 24 02:15 .ipynb_checkpoints
-rw-r--r-- 1 ubuntu ubuntu      311 Jun 24 00:09 Pipfile
-rw-r--r-- 1 ubuntu ubuntu   124837 Jun 24 00:10 Pipfile.lock
-rw-r--r-- 1 ubuntu ubuntu  1915228 Jun 24 01:44 dict_vectorizer.bin
-rw-r--r-- 1 ubuntu ubuntu    15715 Jun 24 02:51 homework4.ipynb
drwxr-xr-x 3 ubuntu ubuntu     4096 Jun 24 01:44 mlartifacts
-rw-r--r-- 1 ubuntu ubuntu   225280 Jun 24 01:55 mlflow.db
drwxr-xr-x 4 ubuntu ubuntu     4096 Jun 24 01:47 mlruns
-rw-r--r-- 1 ubuntu ubuntu    17376 Jun 24 02:19 model.bin
-rw-r--r-- 1 ubuntu ubuntu 68641704 Jun 24 02:50 output_file
-rw-r--r-- 1 ubuntu ubuntu     1302 Jun 24 02:13 predict.py
-rw-r--r-- 1 ubuntu ubuntu    64256 Jun 24 02:15 random-forest.ipynb
-rw-r--r-- 1 ubuntu ubuntu      480 Jun 24 02:01 test.py
